In [52]:
import torch
from skimage import io
import skimage.color
import numpy as np
from model.unet import UNet
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn

device = torch.device('cuda:0')

def load_bw_image(path):
    # Crop so that both dimensions are multiples of 32
    original = io.imread(path)[4:-4]
    L_channel = (original[:, :, 0] - 128) / 128
    L_channel = np.expand_dims(L_channel, axis=[0, 1]).astype(np.float32)
    return L_channel

def load_color_image(path):
    # Crop so that both dimensions are multiples of 32
    original = io.imread(path)[4:-4]
    image = skimage.color.rgb2lab(original)
    L_channel = (image[:, :, 0] - 50) / 50
    L_channel = np.expand_dims(L_channel, axis=[0, 1]).astype(np.float32)
    return original, L_channel

In [2]:
model_path = '/mnt/Research/ShanghaiProject/code/checkpoint/lr0.0001_2020-05-17-01-03-46/epoch1_iter9999.pt'
model = torch.nn.DataParallel(UNet().to(device))

checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [53]:
image_path = '/mnt/Research/ShanghaiProject/test-dataset/test/1166.png'
image2_path = '/mnt/Research/ShanghaiProject/test-dataset/test/1137.png'
image3_path = '/mnt/Research/ShanghaiProject/test-dataset/bw/992.png'

original_image, L_channel = load_color_image(image2_path)
# L_channel = load_bw_image(image3_path)
# print(f'Image shape: {original_image.shape}')
print(f'L channel shape: {L_channel.shape}')

L channel shape: (1, 1, 352, 480)


In [54]:
resnet = models.resnet34(pretrained=True)._modules
resnet_input = torch.tensor(original_image.transpose((2, 0, 1))).unsqueeze(0).float()

resnet_output = resnet_input
for layer in list(resnet)[:-2]:
    resnet_output = resnet[layer](resnet_output)
    print(resnet_output.shape)


torch.Size([1, 64, 176, 240])
torch.Size([1, 64, 176, 240])
torch.Size([1, 64, 176, 240])
torch.Size([1, 64, 88, 120])
torch.Size([1, 64, 88, 120])
torch.Size([1, 128, 44, 60])
torch.Size([1, 256, 22, 30])
torch.Size([1, 512, 11, 15])


In [55]:
deconv = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
print(deconv(resnet_output).shape)

torch.Size([1, 256, 22, 30])


In [ ]:
output = model(torch.tensor(L_channel).to(device))
print(f'Predictions shape: {output.shape}')

In [56]:
ab_channels = output.squeeze().cpu().detach().numpy()
Lab = np.concatenate([L_channel[0], ab_channels], axis=0)
Lab = Lab.transpose((1, 2, 0))
print(f'Lab shape: {Lab.shape}')

# Lab = (Lab * [50, 128, 128] + [50, 0, 0])
Lab = (Lab * [128, 128, 128] + [128, 0, 0])
print(f'Mean in L channel: {Lab[:, :, 0].mean()}')
print(f'Mean in a channel: {Lab[:, :, 1].mean()}')
print(f'Mean in b channel: {Lab[:, :, 2].mean()}')

rgb_output = skimage.color.lab2rgb(Lab)

plt.imshow(rgb_output)


RuntimeError: CUDA error: unspecified launch failure